In [21]:
!pip install transformers bert-for-tf2


In [22]:
pip install --upgrade transformers bert-for-tf2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

In [2]:
# Instantiate the Model and Tokenizer
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = TFBertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-07-21 21:06:11.650767: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-21 21:06:11.651015: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [3]:
# Define the Sentiment Analysis Function
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='tf', max_length=512, truncation=True)
    result = model(tokens)
    return int(tf.argmax(result.logits, axis=1)[0])

In [4]:
df = pd.read_csv('one_row.csv')

In [5]:
df

,index,date,comments,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,percent_chnage_x,...,High_y,Low_y,Close_y,Adj Close_y,Volume_y,percent_chnage_y,Jump_y,Big_Jump_y,Drop_y,Big_Drop_y
0,1,2022-07-14,This week's [Earnings Thread](https://www.redd...,3763.98999,3796.409912,3721.560059,3790.379883,3790.379883,4199690000,0.701115,...,11279.96973,11005.92969,11251.19043,11251.19043,4481070000,0.896589,1,0,0,0


In [6]:
# Split comments into sentences
df['bert_sentiment'] = df['comments'].str.split('.').apply(lambda sentences: [sentiment_score(sentence) for sentence in sentences])


In [7]:
# Save the DataFrame with sentiment scores to a new CSV file
df.to_csv('bert_sentiment.csv', index=False)